In [73]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

In [74]:
FullLabelPath = 'egd_trainLabels_extensive2.txt'
FullLabelFile = pd.read_csv(FullLabelPath,names=['ID','label','extra'],sep='\t',low_memory=False)

In [75]:
FullLabelFile

,ID,label,extra
0,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0
1,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0
2,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0
3,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0
4,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0
...,...,...,...
42195,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0
42196,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0
42197,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0
42198,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0


In [76]:
FullLabelFile['label'].value_counts()

label
1     8500
9     8375
0     6400
2     5750
13    5250
3     5075
10    2225
17     625
Name: count, dtype: int64

In [77]:
amountsPerClass = [1, 5, 10, 20]

In [78]:
def extractNiftiFilepathAndSlicenum(df):
    ID = df['ID']
    split = ID.rsplit('__s', 1)
    NiftiPath = split[0]
    NiftiPath = NiftiPath.replace('NIFTI_SLICES', 'NIFTI') + '.nii.gz'
    slicenum = int(split[1].split('.nii.gz')[0])
    return pd.Series({'NiftiPath': NiftiPath, 'slicenum': slicenum})

In [79]:
FilepathsAndSlicenums = FullLabelFile.apply(extractNiftiFilepathAndSlicenum, axis=1)

In [80]:
FullLabelFile = FullLabelFile.merge(FilepathsAndSlicenums, left_index=True, right_index=True)

In [86]:
FullLabelFile

,ID,label,extra,NiftiPath,slicenum
0,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,17
1,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,1
2,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,10
3,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,13
4,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,15
...,...,...,...,...,...
42195,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,4
42196,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,3
42197,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,19
42198,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,17


In [93]:
for numTrainSamples in amountsPerClass:
    extension = os.path.splitext(FullLabelPath)
    basename = extension[0]
    extension = extension[1]
    df = pd.DataFrame()
    for label in FullLabelFile['label'].unique():
        outname = f'{basename}_s{numTrainSamples}{extension}'
        LabelFrame = FullLabelFile[FullLabelFile['label']==label]
        NiftiNames = LabelFrame['NiftiPath'].unique()
        train, test = train_test_split(NiftiNames, train_size=numTrainSamples, shuffle=True, random_state=42)
        for t in train:
            if df.empty:
                df = LabelFrame[LabelFrame['NiftiPath']==t]
            else:
                temp = LabelFrame[LabelFrame['NiftiPath']==t]
                df = pd.concat([df, temp],axis=0)
        df[['ID','label','extra']].to_csv(outname, index=False, header=False, sep='\t')

In [94]:
df

,ID,label,extra,NiftiPath,slicenum
22300,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_15...,23
22301,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_15...,9
22302,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_15...,14
22303,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_15...,12
22304,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_15...,0
...,...,...,...,...,...
13120,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,14
13121,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,11
13122,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,6
13123,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,5


In [95]:
control = 'egd_trainLabels_extensive2_s10.txt'
ControlLabelFile = pd.read_csv(control,names=['ID','label','extra'],sep='\t',low_memory=False)

In [96]:
ControlLabelFile['label'].value_counts()

label
1     275
0     250
10    250
2     250
3     250
17    250
13    250
9     250
Name: count, dtype: int64